In [1]:
import torch

x = torch.tensor([[1,2,3], [4,5,6],[7,8,9]])
print(x)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [2]:
import torch
import numpy
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

In [3]:
n_dim = 2
x_train, y_train = make_blobs(n_samples=80, n_features=n_dim, \
centers=[[1,1],[-1,-1],[1,-1],[-1,1]], shuffle=True, cluster_std=0.3)
x_test, y_test = make_blobs(n_samples=20, n_features=n_dim, \
centers=[[1,1],[-1,-1],[1,-1],[-1,1]], shuffle=True, cluster_std=0.3)

In [4]:
def label_map(y_, from_, to_):
    y = numpy.copy(y_)
    for f in from_:
        y[y_ == f] = to_
    return y

y_train = label_map(y_train, [0, 1], 0)
y_train = label_map(y_train, [2, 3], 1)
y_test = label_map(y_test, [0, 1], 0)
y_test = label_map(y_test, [2, 3], 1)

In [5]:
x_train = torch.FloatTensor(x_train)
print(x_train.shape)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

torch.Size([80, 2])


In [6]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size): # NN structure
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, input_tensor): # NN behavior
        linear1 = self.linear_1(input_tensor)
        relu = self.relu(linear1)
        linear2 = self.linear_2(relu)
        output = self.sigmoid(linear2)
        return output

회귀 / 분류 모델의 차이
sigmoid -> binary_cross_entropy 사용
softmax -> cross_entropy

In [7]:
model = NeuralNet(2, 5)
learning_rate = 0.03
criterion = torch.nn.BCELoss() # Binary cross entropy
epochs = 2000
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # SGD

In [8]:
for epoch in range(epochs):
    model.train() # train mode
    optimizer.zero_grad() # reset the gradients of model parameters
    train_output = model(x_train)
    train_loss = criterion(train_output.squeeze(), y_train)
    if epoch % 100 == 0:
        print('Train loss at {} is {}'.format(epoch, train_loss.item()))
    train_loss.backward()
    optimizer.step()

Train loss at 0 is 0.7154176831245422
Train loss at 100 is 0.6449922323226929
Train loss at 200 is 0.5660666823387146
Train loss at 300 is 0.4664331078529358
Train loss at 400 is 0.3495132327079773
Train loss at 500 is 0.24527709186077118
Train loss at 600 is 0.16971400380134583
Train loss at 700 is 0.12110352516174316
Train loss at 800 is 0.0908563956618309
Train loss at 900 is 0.07129959762096405
Train loss at 1000 is 0.05791493132710457
Train loss at 1100 is 0.04839223250746727
Train loss at 1200 is 0.0412762276828289
Train loss at 1300 is 0.035811398178339005
Train loss at 1400 is 0.03149789944291115
Train loss at 1500 is 0.028021156787872314
Train loss at 1600 is 0.025169452652335167
Train loss at 1700 is 0.022795137017965317
Train loss at 1800 is 0.020792633295059204
Train loss at 1900 is 0.0190841406583786


In [9]:
model.eval() # evaluation mode
test_loss = criterion(torch.squeeze(model(x_test)), y_test)
print('After Training, test loss is {}'.format(test_loss.item()))

After Training, test loss is 0.030878927558660507


In [10]:
torch.save(model.state_dict(), './model.pt')
print('state_dict format of the model: {}'.format(model.state_dict()))

state_dict format of the model: OrderedDict({'linear_1.weight': tensor([[ 0.4631, -0.2895],
        [-1.6329, -1.6336],
        [ 1.3159, -1.3984],
        [-1.3586,  1.3008],
        [ 1.4161,  1.5173]]), 'linear_1.bias': tensor([-0.1403, -0.0144,  0.0104,  0.0200, -0.1874]), 'linear_2.weight': tensor([[ 0.4013, -2.2527,  1.6661,  1.8111, -1.9711]]), 'linear_2.bias': tensor([0.7467])})


In [12]:
new_model = NeuralNet(2, 5)
new_model.load_state_dict(torch.load('./model.pt'))
new_model.eval()
print('The prob. that [-1,1] has label 1 is {}'.format(new_model(torch.FloatTensor([-1,1])).item()))

The prob. that [-1,1] has label 1 is 0.99631267786026
